## Model Evaluation with Mlflow

In [2]:
import os

In [3]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai\\research\\Food image Classification'

In [6]:
os.chdir("../")

In [7]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai'

In [8]:
os.environ['MLFLOW_TRACKING_URI']= "https://dagshub.com/itzmeakashps/Healthy.ai.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']= "itzmeakashps"
os.environ['MLFLOW_TRACKING_PASSWORD']= "d587244e64e62ea9ab9a724d252a21ca83c524fe"

MLFLOW_TRACKING_URI=https://dagshub.com/itzmeakashps/Healthy.ai.mlflow \
MLFLOW_TRACKING_USERNAME=itzmeakashps \
MLFLOW_TRACKING_PASSWORD=d587244e64e62ea9ab9a724d252a21ca83c524fe \
python script.py

In [9]:
import tensorflow as tf 


In [10]:
#load the model

model = tf.keras.models.load_model("artifacts/food_image_classification/training/model.h5")


## Prepare the Entity

In [11]:
from pathlib import  Path
from dataclasses import dataclass


@dataclass(frozen=True)
class FoodEvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size: list
    params_batch_size: int
    

## Configuration Manager in src config

In [13]:
from Healthyai.constants import *
from Healthyai.utils.common import read_yaml,create_directories, save_json


In [14]:
class ConfigurationManger:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts])
        
    def get_food_evaludation_config(self) -> FoodEvaluationConfig:
        food_eval_config = FoodEvaluationConfig(
            path_of_model="artifacts/food_image_classification/training/model.h5",
            training_data= "artifacts/food_image_classification/data_ingestion/food classification",
            mlflow_uri= "https://dagshub.com/itzmeakashps/Healthy.ai.mlflow",
            all_params = self.params.VGG16,
            params_image_size= self.params.VGG16.IMAGE_SIZE,
            params_batch_size= self.params.VGG16.BATCH_SIZE
        )
        return food_eval_config
    

## Components 


In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\pydantic\_internal\_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [16]:
class FoodModelEvaluation:
    def __init__(self, config: FoodEvaluationConfig):
        self.config = config
        
    
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split= 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
              **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
            
        )
        
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"),data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )   
            
            # Model registry does not work with file score
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")
                
                     
        

In [17]:
#pipeline

try:
    config = ConfigurationManger()
    eval_config = config.get_food_evaludation_config()
    evaluation = FoodModelEvaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-15 18:23:02,858: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-15 18:23:02,860: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-15 18:23:02,860: INFO: common: Created directory at : artifacts]
Found 2970 images belonging to 11 classes.

[2023-12-15 18:23:03,558: WARNING: module_wrapper: From c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
186/186 [==============================] - 95s 511ms/step - loss: 12.1136 - accuracy: 0.5044
[2023-12-15 18:24:38,807: INFO: common: json file saved at: scores.json]


2023/12/15 18:24:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\akash\AppData\Local\Temp\tmpfvlb86dz\model\data\model\assets
[2023-12-15 18:24:41,693: INFO: builder_impl: Assets written to: C:\Users\akash\AppData\Local\Temp\tmpfvlb86dz\model\data\model\assets]


c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/12/15 18:25:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.
